In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


## Required Packages

In [ ]:
import sys, os
import math
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import matplotlib as mpl
import matplotlib.pyplot as plt
import skimage

## Version check

In [ ]:
print('Python: {}'.format(sys.version))
print('Numpy: {}'.format(np.__version__))
print('Pandas: {}'.format(pd.__version__))
print('OpenCV: {}'.format(cv2.__version__))
print('Tensorflow: {}'.format(tf.__version__))
print('Matplotlib: {}'.format(mpl.__version__))
print('Scikit-Image: {}'.format(skimage.__version__))

### Peak-Signal-to-Noise Ratio (PSNR)

In [ ]:
def psnr(target, ref):
    # Assume target is RGB/BGR image
    target_data = target.astype(np.float32)
    ref_data = ref.astype(np.float32)

    diff = ref_data - target_data
    diff = diff.flatten('C')

    rmse = np.sqrt(np.mean(diff ** 2.))

    return 20 * np.log10(255. / rmse)

### Mean Squared Error (MSE)

In [ ]:
def mse(target, ref):
    target_data = target.astype(np.float32)
    ref_data = ref.astype(np.float32)
    err = np.sum((target_data - ref_data) ** 2)

    err /= np.float(target_data.shape[0] * target_data.shape[1])
    return err

### Strucutural Similarity (SSIM)



In [ ]:
from skimage.metrics import structural_similarity as ssim

After we defined our metrics for measuring image quality, we need to combine whole metrics in one metric.

In [ ]:
def compare_frames(target, ref):
    scores = []
    scores.append(psnr(target, ref))
    scores.append(mse(target, ref))
    scores.append(ssim(target, ref, multichannel=True))
    return scores

In [ ]:
%cd /content/gdrive/MyDrive/p4p/

Path to videos

In [ ]:
video_path = '/content/gdrive/'          # real video path (.mp4)
generated_video_path = '/content/gdrive'       # video path that wants to be evaluated (.mp4)

In [ ]:
# Open original video
original_video = cv2.VideoCapture(video_path)

# Open resized video
generated_video = cv2.VideoCapture(generated_video_path)

# Get the number of frames in the video
num_frames = int(original_video.get(cv2.CAP_PROP_FRAME_COUNT))

psnr_values = []
mse_values = []
ssim_values = []

In [ ]:
# Iterate over each frame
for _ in range(num_frames):
    # Read frames from the original video
    ret_o, frame_o = original_video.read()

    # Read frames from the resized video
    ret_r, frame_r = generated_video.read()

    if not ret_o or not ret_r:
        break

    # Resize frames to a consistent size
    frame_o = cv2.resize(frame_o, (frame_r.shape[1], frame_r.shape[0]))

    # Compare frames and calculate scores
    scores = compare_frames(frame_o, frame_r)

    # Append scores to the lists
    psnr_values.append(scores[0])
    mse_values.append(scores[1])
    ssim_values.append(scores[2])

# Calculate average scores
avg_psnr = np.mean(psnr_values)
avg_mse = np.mean(mse_values)
avg_ssim = np.mean(ssim_values)

print("Average PSNR:", avg_psnr)
print("Average MSE:", avg_mse)
print("Average SSIM:", avg_ssim)

# Release video captures
original_video.release()
generated_video.release()

### Frechet inception Distance (FID)

In [ ]:
!pip install pytorch-fid

In [ ]:
!python -m pytorch_fid /content/gdrive/ /content/gdrive/         # path of real source video and video wants to get evaluated (.mp4)